# 1. Introduction and Foundational Knowledge
## A. Introduction

### Hydrogen molecular ion

This Jupyter Notebook is a guide through obtaining the molecular orbitals of the quantum system of $\text{H}_{2}^{+}$ by directly solving the Schrödinger equation, using Python (with modules such as NumPy and SciPy). This work is derived from the following resources.

- Grivet, J.-P. The Hydrogen Molecular Ion Revisited. Journal of Chemical Education, 2002, 79, 127. https://doi.org/10.1021/ed079p127.
- Johnson, J. L. Visualization of Wavefunctions of the Ionized Hydrogen Molecule. Journal of Chemical Education, 2004, 81, 1535. https://doi.org/10.1021/ed081p1535.1.

The sources should help when viewed alongside this Jupyter Notebook. The notations used are that of Grivet[<sup>1</sup>](#fn1).

Prior knowledge of introductory quantum mechanics — particularly about the hydrogen atomic orbitals — is required. The following textbooks will be helpful with this regard.

For an in-depth review on quantum chemistry:

- *Quantum Chemistry* by Ira N. Levine

For an approachable yet complete review on quantum chemistry:

- *Physical Chemistry: A Molecular Approach* by McQuarrie and Simon

For an introduction to atomic orbitals and molecular orbitals:

- *Principles of Modern Chemistry* by David W. Oxtoby

As for the required background on mathematics, an elementary understanding of differential equations will suffice.

### Python information
The following modules are *required*.

- NumPy
- SciPy for solving differential equations
- Matplotlib for 2D graphing

Other modules used are

- Plotly for 3D graphing (required for viewing 3D plots, images of which are already generated)
- scikit-image (for generating 3D meshes, which are already generated)

## B. Background Quantum Mechanics

### Atomic orbitals

The process of obtaining molecular orbitals of the hydrogen molecular ion is similar to process of obtaining atomic orbitals of hydrogen. Therefore, before we begin with molecular orbitals, here is a quick refresher on atomic orbitals.

The wavefunction of the one-electron system of the hydrogen atom can be solved exactly (if only the coulombic attraction is accounted in the Hamiltonian). For the rest of the notebook, the Born-Oppenheimer approximation is used (in other words, the proton is assumed to be fixed at the origin for convenience).

$$ \hat{H}=-\frac{\hbar}{2m_{e}}\nabla^{2}-\frac{e^{2}}{4\pi\varepsilon_{0}r} $$

A new system of units, called Hartree atomic units, is introduced to simplify calculations.

| Property     | Atomic unit                   | Notes    |
|--------------|-------------------------------|---------------------|
| mass         | $m_e$, the mass of an electron| equal to $9.11 \times 10^{-31} \, \text{kg}$ |
| charge       | $e$, the elementary charge    | equal to $1.60 \times 10^{-19} \, \text{C}$  |
| angular momentum or action | $\hbar$, the Planck constant over $2\pi$ |     |
| distance     | $a_0$, Bohr radius            | equal to $5.29 \times 10^{-11} \, \text{m}$  |
| energy       | $E_h=\frac{m_{e}e^{4}}{16\pi^{2}\varepsilon_{0}^{2}\hbar^{2}}$, the Hartree | equal to $27.2 \; \text{eV}$ |
| permittivity | $4\pi\varepsilon_{0}$         |    |

Using the new system of units, the Hamiltonian can be further simplified.

$$ \hat{H}=-\frac{1}{2}\nabla^{2}-\frac{1}{r} $$

To exploit the spherical symmetry of the Hamiltonian, we use spherical coordinates $\left ( r, \theta, \phi \right )$ as shown below.

![Spherical Coordinates](img/spherical_coords.png)

Note that the bound for each of the coordinates are $0 \leq r\, ,\; 0 \leq \theta \leq \pi\, ,\; 0 \leq \phi < 2\pi$.

The Laplacian operator $\nabla^{2}$ in spherical coordinates is

$$\nabla^{2}=\frac{1}{r^2}\frac{\partial }{\partial r}\left ( r^{2}\frac{\partial }{\partial r} \right )+\frac{1}{r^{2}\sin \theta}\frac{\partial }{\partial \theta}\left ( \sin \theta \frac{\partial }{\partial \theta} \right ) + \frac{1}{r^{2}\sin^{2}\theta}\frac{\partial^2 }{\partial \phi^2}$$

With this Hamiltonian and the boundary condition $ \psi \to 0 \text{ as }r \to \infty $, the time-independent Schrödinger equation $\hat{H}\psi = E\psi$ can be solved through separation of variables. This amounts to solving three differential equations with three different boundary conditons, each with respect to $\phi, \theta, r$. The final wave function is formed by combining the three solutions.

$$ \psi(r, \theta, \phi) = R(r)\,  Y(\theta, \phi) = R(r) \, \Theta(\theta)\, \Phi(\phi) $$

Note that from this point on, we deal with *unnormalized* radial/angular functions and wavefunctions as the normalization step can happen at the very end, either analytically or numerically.

### The radial equation and the angular equations
With the Laplacian operator $\nabla^{2}$ in spherical coordinates and $\hat{H}\psi = E\psi$, the following equations are derived.

$$-\frac{1}{2}\left [\frac{1}{r^2}\frac{\partial }{\partial r}\left ( r^{2}\frac{\partial \psi}{\partial r} \right )+\frac{1}{r^{2}\sin \theta}\frac{\partial }{\partial \theta}\left ( \sin \theta \frac{\partial \psi}{\partial \theta} \right ) + \frac{1}{r^{2}\sin^{2}\theta}\frac{\partial^2 \psi }{\partial \phi^2}  \right ] -\frac{1}{r}\,  \psi(r, \theta, \phi)=E\, \psi(r, \theta, \phi)$$

We multiply through by $2r^2$, substitute $\psi$ with $R(r)\,  Y(\theta, \phi)$, and at last, divide by $R(r)\,  Y(\theta, \phi)$.

$$\frac{1}{R(r)}\left [  \frac{d }{d r} \left  (r^{2}\frac{d R}{d r}  \right ) + 2r^{2} \left ( \frac{1}{r} + E \right ) R(r) \right ]  = - \frac{1}{Y(\theta, \phi)} \left [ \frac{1}{\sin \theta} \frac{\partial }{\partial \theta}\left ( \sin \theta \frac{\partial Y}{\partial \theta} \right ) + \frac{1}{\sin^{2}\theta } \frac{\partial^2 Y}{\partial \psi^2} \right ] = \beta$$

The left side of the equality depends only on $r$, while the right side of the equality depends only on $\theta$ and $\phi$. Thus, both sides must be independent from all variables: a constant, with respect to either $r, \theta$, or $\phi$. This new constant is denoted as $\beta$.

#### Angular equations

We focus our attention to the right side of the equality. Multiplying the equation through by $\sin^{2} \theta$ and rearranging it yields

$$\sin \theta \frac{\partial }{\partial \theta}\left ( \sin \theta \frac{\partial Y}{\partial \theta} \right ) + \frac{\partial^2 Y}{\partial \phi^2} + \left (\beta \sin^{2}\theta  \right ) Y = 0$$

Separating the $\theta$ dependent terms and $\phi$ dependent terms is now possible. Substituting $Y(\theta, \phi)$ with $\Theta(\theta) \, \Phi(\phi)$ and then multiplying the equation by $\Theta(\theta) \, \Phi(\phi)$ results in the following.

$$\frac{\sin \theta}{\Theta (\theta)}\frac{d }{d \theta}\left ( \sin \theta \frac{d \Theta }{d \theta} \right ) + \beta \sin^{2}\theta = -\frac{1}{\Phi (\phi)}\frac{d^2 \Phi }{d \phi^2} = m^2$$

The left side of the equality depends only on $\theta$, while the right side of the equality depends only on $\theta$ and $\phi$. Again, both sides can be expressed as a constant with respect to either $\theta$ and $\phi$, which shall be denoted as $m^2$.

With the boundary condition $\Phi(0) = \Phi(2\pi)$, the equality $-\frac{1}{\Phi (\phi)}\frac{d^2 \Phi }{d \phi^2} = m^2$ yields the following yet-to-be-normalized solution.

$$ \Phi_{m}(\phi)=e^{im\phi}\qquad m=0 ,\; \pm1,\; \pm 2,\; \cdots  $$

The equation $\frac{\sin \theta}{\Theta (\theta)}\frac{d }{d \theta}\left ( \sin \theta \frac{d \Theta }{d \theta} \right ) + \beta \sin^{2}\theta = m^2$ is a bit harder to solve, but solving it yields the associated Legendre polynomial. Consult your physical/quantum chemistry textbook if necessary[<sup>2</sup>](#fn2). 

The solution for $\Theta (\theta)$ is as follows.

$$\Theta (\theta) = P_{l}^{\left | m \right |}(\cos \theta) \text{ where } P_{l}^{\left | m \right |} \text{ is the associated Legendre polynomial}$$

Note that during the solving process, it is revealed that $\beta = l(l+1) \geq m^2 $ (thus $\left | m  \right | \leq l$).

#### Radial equations

By substituting $\beta$ with $l(l+1)$ in one of our previous equations and rearranging some terms, we are left with the following equation.

$$-\frac{1}{2r^2}\frac{d }{d r} \left  (r^{2}\frac{d R}{d r}  \right ) + \left [ \frac{l(l+1)}{2r^{2}} -\frac{1}{r} -E  \right ]R(r) = 0$$

The process of solving the equation above will not be treated in detail in this notebook. Nevertheless, during the such process, the quantization of energy $E$ is revealed, with each positive integer $n = 1, 2, 3, \cdots$ corresponding to a particular energy level.

$$E_n = -\frac{1}{2}\frac{1}{n^2} \text{ in atomic units}$$

It is also shown that $n$, the principle quantum number, and $l$ the angular momentum quantum number must satisfy the following relationship.

$$0 \leq l \leq n - 1$$

Finally, the radial equation is given as an expression containing the associated Laguerre polynomial.

$$R_{nl}(r)=r^{l}e^{-r/n}\left [L_{n+l}^{2l+1}\left (\frac{2r}{n}  \right )  \right ] \text{ where } L_{n+l}^{2l+1}(x) \text{ is the associated Laguerre polynomial}$$

### Real wavefunctions

Although the full wavefunction solutions are hereby obtained, there is one step left to obtain the familiar atomic orbitals used by some physicists and most chemists. Because the $\phi$-dependent term $\Phi_{m}(\phi)=e^{im\phi}$ is complex-valued, the wavefunctions are also complex-valued (for $m \neq 0$). These can be mitigated by using **combinations** of $\Phi_{m}(\phi)$ in the place of  $\Phi_{m}(\phi)$.

Note: all $\Phi(\phi)$ are not normalized.

For the case of $l=1$ $(m = -1,\, 0,\, 1)$

$
\begin{align}
\quad \Phi_{z}(\phi) = \Phi_{0}(\phi) = e^{0i\phi} = 1
\end{align}
$

$
\begin{align}
\quad \Phi_{x}(\phi) = \frac{\Phi_{1}(\phi)+\Phi_{-1}(\phi)}{2}  = \frac{e^{i\phi}+e^{-i\phi}}{2} = \cos \phi
\end{align}
$

$
\begin{align}
\quad \Phi_{y}(\phi) = \frac{\Phi_{1}(\phi)-\Phi_{-1}(\phi)}{2i}  = \frac{e^{i\phi}-e^{-i\phi}}{2i} = \sin \phi 
\end{align}
$

For the case of $l=2$ and $m = -2,\,-1, \, 0, \, 1, \, 2$ (other possible combinations also exist)

$
\begin{align}
\quad \Phi_{z^2}(\phi) = \Phi_{0}(\phi) = e^{0i\phi} = 1
\end{align}
$

$
\begin{align}
\quad \Phi_{xz}(\phi) = \frac{\Phi_{1}(\phi)+\Phi_{-1}(\phi)}{2}  = \frac{e^{i\phi}+e^{-i\phi}}{2} = \cos \phi
\end{align}
$

$
\begin{align}
\quad \Phi_{yz}(\phi) = \frac{\Phi_{1}(\phi)-\Phi_{-1}(\phi)}{2i}  = \frac{e^{i\phi}-e^{-i\phi}}{2i} = \sin \phi
\end{align}
$

$
\begin{align}
\quad \Phi_{x^{2}-y^{2}}(\phi) = \frac{\Phi_{2}(\phi)+\Phi_{-2}(\phi)}{2}  = \frac{e^{2i\phi}+e^{-2i\phi}}{2} = \cos 2\phi
\end{align}
$

$
\begin{align}
\quad \Phi_{xy}(\phi) = \frac{\Phi_{2}(\phi)-\Phi_{-2}(\phi)}{2i}  = \frac{e^{2i\phi}-e^{-2i\phi}}{2i} = \sin 2\phi
\end{align}
$

## C. What to Expect From the Hydrogen Molecular Ion

### Challenges of solving the hydrogen molecular ion

The hydrogen molecular ion (HMI) consists of one electron and two protons (with an equilibrium internuclear distance of approximately $2a_0$), instead of one electron and proton of the hydrogen atom. Although the HMI is technically a three-body problem (which cannot be solved exactly), the Born-Oppenheimer approximation allows the quantum system to be solvable.

Even with the Born-Oppenheimer approximation, obtaining the exact analytical solution is incredibly demanding in mathematical prowess and, more importantly, patience. However, through differential equation solvers such as the one included in SciPy, we can obtain the nearly-exact solution to HMI. 

### Why solve the hydrogen molecular ion

During the process of manipulating the time-independent Schrodinger equation, numerically solving the required parameters, and examining the nearly-exact solution, a deeper insight into chemical bonding can be formed.

As the HMI consists of one electron and two protons, this quantum system can also be thought of as the problem below.

> Imagine a stable Helium-2 ion $^{2}\text{He}^{+}$ with one electron. Initially, the wavefunction of the electron is one of the typical hydrogen-like atomic orbitals. 
>
>As you forcefully tear the two protons of the nucleus apart, the electron wavefunction slowly distorts to reflect the changing Hamiltonian. When the two protons are separated by a distance of $2a_0$, the electron wave function is that of the familiar HMI treated in physical/quantum chemistry textbooks. 
>
>At last, as the two protons continue to be separated, this system now closely resembles that of the infinitely separated proton and neutral hydrogen atom (again, think of hydrogen atomic orbitals).

With the perspective offered above, it is clear that the hydrogen atom and HMI are deeply related to each other. This relationship will be investigated in the following notebooks as we uncover the HMI.

Finally, the benefits of other methods, such as LCAO-MO, will be very apparent as the HMI is solved with the aforementioned, computationaly intensive method.

## Footnotes

<span id="fn1">1. Grivet uses atomic units, with the unit of energy being the Rydberg (= 0.5 Hartrees). This is unlike other literature and most textbooks, where the unit of energy is the Hartree. This notebook uses the latter convention.</span>

<span id="fn2">2. For such physical chemistry textbook, a recommendation of mine is *Physical Chemistry: A Molecular Approach* by McQuarrie and Simon.</span>